# Detect Model Bias using `smclarify`


## Amazon Science: _[How Clarify helps machine learning developers detect unintended bias](https://www.amazon.science/latest-news/how-clarify-helps-machine-learning-developers-detect-unintended-bias)_ 

[<img src="img/amazon_science_clarify.png"  width="100%" align="left">](https://www.amazon.science/latest-news/how-clarify-helps-machine-learning-developers-detect-unintended-bias)

# Terminology

* **Bias**: 
An imbalance in the training data or the prediction behavior of the model across different groups, such as age or income bracket. Biases can result from the data or algorithm used to train your model. For instance, if an ML model is trained primarily on data from middle-aged individuals, it may be less accurate when making predictions involving younger and older people.

* **Bias metric**: 
A function that returns numerical values indicating the level of a potential bias.

* **Bias report**:
A collection of bias metrics for a given dataset, or a combination of a dataset and a model.

* **Label**:
Feature that is the target for training a machine learning model. Referred to as the observed label or observed outcome.

* **Positive label values**:
Label values that are favorable to a demographic group observed in a sample. In other words, designates a sample as having a positive result.

* **Negative label values**:
Label values that are unfavorable to a demographic group observed in a sample. In other words, designates a sample as having a negative result.

* **Facet**:
A column or feature that contains the attributes with respect to which bias is measured.

* **Facet value**:
The feature values of attributes that bias might favor or disfavor.

# Posttraining Bias Metrics
https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-measure-post-training-bias.html

* **Difference in Positive Proportions in Predicted Labels (DPPL)**:
Measures the difference in the proportion of positive predictions between the favored facet a and the disfavored facet d.

* **Disparate Impact (DI)**:
Measures the ratio of proportions of the predicted labels for the favored facet a and the disfavored facet d.

* **Difference in Conditional Acceptance (DCAcc)**:
Compares the observed labels to the labels predicted by a model and assesses whether this is the same across facets for predicted positive outcomes (acceptances).

* **Difference in Conditional Rejection (DCR)**:
Compares the observed labels to the labels predicted by a model and assesses whether this is the same across facets for negative outcomes (rejections).

* **Recall Difference (RD)**:
Compares the recall of the model for the favored and disfavored facets.

* **Difference in Acceptance Rates (DAR)**:
Measures the difference in the ratios of the observed positive outcomes (TP) to the predicted positives (TP + FP) between the favored and disfavored facets.

* **Difference in Rejection Rates (DRR)**:
Measures the difference in the ratios of the observed negative outcomes (TN) to the predicted negatives (TN + FN) between the disfavored and favored facets.

* **Accuracy Difference (AD)**:
Measures the difference between the prediction accuracy for the favored and disfavored facets.

* **Treatment Equality (TE)**:
Measures the difference in the ratio of false positives to false negatives between the favored and disfavored facets.

* **Conditional Demographic Disparity in Predicted Labels (CDDPL)**:
Measures the disparity of predicted labels between the facets as a whole, but also by subgroups.

* **Counterfactual Fliptest (FT)**:
Examines each member of facet d and assesses whether similar members of facet a have different model predictions.


In [ ]:
!pip install -q smclarify==0.1

In [ ]:
from smclarify.bias.report import *
from smclarify.util.dataset import Datasets, german_lending_readable_values
from typing import Dict
from collections import defaultdict
import pandas as pd

In [ ]:
import boto3
import sagemaker
import pandas as pd
import numpy as np

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format='retina'

# Test data for bias

We created test data in JSONLines format to match the model inputs. 

In [ ]:
data = {
    "star_rating": [1, 2, 3, 4, 5],
    "review_body": ["Worst ever", "Expected more", "Its ok", "I like it", "I love it"],
    "product_category": ["Gift Card", "Gift Card", "Gift Card", "Digital_Software", "Digital_Software"],
    "star_rating_predicted": [1, 2, 3, 4, 5],
}

In [ ]:
df = pd.DataFrame(data, columns=["star_rating", "review_body", "product_category", "star_rating_predicted"])
print(df)

## Convert data columns into `categorical` data type required for Clarify

In [ ]:
# df['star_rating'] = df['star_rating'].astype('category')

In [ ]:
# df['star_rating_predicted'] = df['star_rating_predicted'].astype('category')

In [ ]:
# df['product_category'] = df['product_category'].astype('category')

# Configure Clarify

In [ ]:
facet_column = FacetColumn(name="product_category", sensitive_values=["Gift Card"])

In [ ]:
label_column = LabelColumn(name="star_rating", data=df["star_rating"], positive_label_values=[5, 4])

In [ ]:
predicted_label_column = LabelColumn(
    name="star_rating_predicted", data=df["star_rating_predicted"], positive_label_values=[5, 4]
)

In [ ]:
post_training_report = bias_report(
    df=df,
    facet_column=facet_column,
    label_column=label_column,
    stage_type=StageType.POST_TRAINING,
    predicted_label_column=predicted_label_column,
    metrics=["DPPL", "DI", "DCA", "DCR", "RD", "DAR", "DRR", "AD", "TE"],
)

# Show Post-Training Bias Report

In [ ]:
from pprint import pprint

pprint(post_training_report)

# Release Resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>